# Zooming Slow-Mo
|Original| Zooming Slow-Mo|
|:-------:|:-------------:|
|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/frame-interpolation/zooming-slow-mo/demo/original.gif' width=250></a>|<a href="url"><img src='https://github.com/sony/nnabla-examples/raw/master/frame-interpolation/zooming-slow-mo/demo/original.gif' width=250></a>|

This example interactively demonstrates [Zooming Slow-Mo](https://arxiv.org/abs/2002.11616), a model for video super-resolution and video frame interpolation.

# Preparation
Let's start by installing nnabla and accessing [nnabla-examples repository](https://github.com/sony/nnabla-examples). If you're running on Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
# May show warnings for newly imported packages if run in Colab default python environment.
# Please click the `RESTART RUNTIME` to run the following script correctly.
# The error message of conflicts is acceptable.
!pip install nnabla-ext-cuda116
!git clone https://github.com/sony/nnabla-examples.git

In [ ]:
%cd nnabla-examples/frame-interpolation/zooming-slow-mo

We also need to install a library to work with videos and images.

In [ ]:
!pip install ffmpeg

Now we define a function which plays videos in Colab. Simply run the following cell.

In [ ]:
from IPython.display import HTML
from base64 import b64encode


def play_video(filename, height=512, width=512):
    mp4 = open(filename, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width={width} height={height} controls>
          <source src={data_url} type="video/mp4">
    </video>""")

Let's also download the pre-trained weight parameters.

In [ ]:
!wget https://nnabla.org/pretrained-models/nnabla-examples/frame-interpolation/zooming-slo-mo/zooming_slo_mo.h5
!wget https://nnabla.org/pretrained-models/nnabla-examples/frame-interpolation/zooming-slo-mo/slo_mo.h5

# Upload Video
Run the following cell to upload your own video. You do not have to upload low resolution, low frame rate video, since we will convert it to low resolution and frame rate later, but try not to upload videos that are too long! We recommend videos of less than 5 seconds.

In [ ]:
from google.colab import files

video = files.upload()

Let's rename the uploaded video file for convenience.

In [ ]:
import os
ext = os.path.splitext(list(video.keys())[-1])[-1]
os.rename(list(video.keys())[-1], "input_video{}".format(ext)) 
input_video = "input_video" + ext

We will now extract frames from the video with low resolution and low frame rate, and subsequently make a low resolution, low frame rate video out of it. 

We will convert it to resolution aspect ratio of width 200 and corresponding height, but if you want to specify your custom ratio, modify the part `scale=200:$new_height` in the cell below. Just make sure that both height and width are multiples of 4, and if aspect ratio is too high, it may result in memory error.

In [ ]:
width=!ffprobe -v error -select_streams v:0 -show_entries stream=width -of csv=p=0 input_video.mp4
height=!ffprobe -v error -select_streams v:0 -show_entries stream=height -of csv=p=0 input_video.mp4
new_height = int(height[0])*200 // int(width[0])
new_height = 4*round(new_height/4)
!mkdir -p frames/input_video
!ffmpeg -i $input_video -vf "fps=10, scale=200:$new_height" frames/input_video/frame_%04d.png
!ffmpeg -i frames/input_video/frame_%04d.png -r 10/1 -y low_res.mp4

# Video super-resolution and frame interpolation

We are ready now!

We will first apply frame interpolation only, and then apply both super-resolution and frame interpolation to see the differences.

First, we run frame interpolation with the following cell, and make a video out of it.

(**Note**: if you run into memory allocation error, please try with shorter video)

In [ ]:
!python inference.py --model slo_mo.h5 --input-dir frames/ --only-slomo
!ffmpeg -i results/input_video/%08d.png -r 24/1 -y slowmo.mp4

Now, let's apply both frame interpolatioon and super-resolution, and also make a video out of it.

In [ ]:
!python inference.py --model zooming_slo_mo.h5 --input-dir frames/
!ffmpeg -i results/input_video/%08d.png -r 24/1 -y zooming_slowmo.mp4

Let's check the results!

First, let's check the low resolution, low frame rate version of the video.

In [ ]:
play_video("low_res.mp4")

OK, it was low resolution, low frame rate video that you proably don't feel like watching.

Now let's see the high frame rate version of it.

In [ ]:
play_video("slowmo.mp4")

Frame rate is higher with frame interpolation, but it's still low resolution.

Finally, let's see the high resolution, high frame rate version!

In [ ]:
play_video("zooming_slowmo.mp4")

Were you able to see the enhancements?

Hope you enjoyed it!